
# Constraint Programming

## Constraint Satisfaction Problem: Sudoku Example

Author: jvachier <br> 
Creation date: March 2023 <br>
Publication date: March 2023 <br>

Three key ingredients <br>
- **Variable**: $x \in X$ <br>
- **Domain**: $\{1,2,3,4,5,6,7,8,9\}$<br>
- **Constraints**: <br>
    - All square 3 x 3 values are different 
    - All row values are different
    - All column values are different

In [103]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np 
import pandas as pd 
from ortools.sat.python import cp_model

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/sudoku/sudoku.csv


In [104]:
def reshape_sudoku(array):
    return np.matrix([np.array(list(array[i:i+9])).astype(np.int) for i in range(0, len(array), 9)])

In [105]:
def solution_sudoku(sudoko_matrix):
    model = cp_model.CpModel()
    x = {}
    for i in range(9):
        for j in range(9):
            if sudoko_matrix[i,j] != 0:
                x[i,j] = sudoko_matrix[i,j]
            else:
                x[i,j] = model.NewIntVar(1,9,"x[{}{}]".format(i, j))
    # Three Constraints
    # All row values are different
    [model.AddAllDifferent([x[i,j] for j in range(9)]) for i in range(9)]
    # All column values are different
    [model.AddAllDifferent([x[i,j] for i in range(9)]) for j in range(9)]
    # All square 3 x 3 values are different
    for row_grid in range(0,9,3):
        for col_grid in range(0,9,3):
            [model.AddAllDifferent([x[row_grid+i,col_grid+j] for j in range(3) for i in range(3)])]
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    result = np.zeros((9,9))
    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        for i in range(9):
            for j in range(9):
                result[i,j] = solver.Value(x[i,j])
    else:
        print('Infeasible solution or Invalid model')
    return result

In [106]:
df_sudoku = pd.read_csv("../input/sudoku/sudoku.csv")

In [107]:
numpy_sudoku_puzzle = df_sudoku['puzzle'].to_numpy()
numpy_sudoku_solution = df_sudoku['solution'].to_numpy()
print(numpy_sudoku_puzzle[0],'\n',numpy_sudoku_solution[0])

070000043040009610800634900094052000358460020000800530080070091902100005007040802 
 679518243543729618821634957794352186358461729216897534485276391962183475137945862


In [108]:
sudoko_matrix0 = reshape_sudoku(numpy_sudoku_puzzle[0])
sudoko_matrix_solution0 = reshape_sudoku(numpy_sudoku_solution[0])

sudoko_matrix10 = reshape_sudoku(numpy_sudoku_puzzle[10])
sudoko_matrix_solution10 = reshape_sudoku(numpy_sudoku_solution[10])

sudoko_matrix100 = reshape_sudoku(numpy_sudoku_puzzle[100])
sudoko_matrix_solution100 = reshape_sudoku(numpy_sudoku_solution[100])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [109]:
result0 = solution_sudoku(sudoko_matrix0)
result10 = solution_sudoku(sudoko_matrix10)
result100 = solution_sudoku(sudoko_matrix100)

In [110]:
print(result0,'\n\n',result10,'\n\n',result100)

[[6. 7. 9. 5. 1. 8. 2. 4. 3.]
 [5. 4. 3. 7. 2. 9. 6. 1. 8.]
 [8. 2. 1. 6. 3. 4. 9. 5. 7.]
 [7. 9. 4. 3. 5. 2. 1. 8. 6.]
 [3. 5. 8. 4. 6. 1. 7. 2. 9.]
 [2. 1. 6. 8. 9. 7. 5. 3. 4.]
 [4. 8. 5. 2. 7. 6. 3. 9. 1.]
 [9. 6. 2. 1. 8. 3. 4. 7. 5.]
 [1. 3. 7. 9. 4. 5. 8. 6. 2.]] 

 [[4. 2. 5. 9. 7. 1. 3. 6. 8.]
 [1. 7. 3. 6. 2. 8. 5. 9. 4.]
 [8. 6. 9. 5. 4. 3. 1. 7. 2.]
 [2. 4. 1. 7. 9. 5. 8. 3. 6.]
 [7. 3. 6. 1. 8. 2. 9. 4. 5.]
 [9. 5. 8. 4. 3. 6. 7. 2. 1.]
 [5. 9. 7. 8. 6. 4. 2. 1. 3.]
 [3. 8. 4. 2. 1. 7. 6. 5. 9.]
 [6. 1. 2. 3. 5. 9. 4. 8. 7.]] 

 [[2. 7. 8. 9. 3. 6. 1. 4. 5.]
 [9. 1. 3. 4. 8. 5. 7. 6. 2.]
 [4. 6. 5. 7. 2. 1. 9. 8. 3.]
 [5. 9. 2. 6. 7. 4. 8. 3. 1.]
 [8. 3. 1. 5. 9. 2. 4. 7. 6.]
 [6. 4. 7. 3. 1. 8. 5. 2. 9.]
 [1. 8. 4. 2. 6. 9. 3. 5. 7.]
 [7. 2. 9. 8. 5. 3. 6. 1. 4.]
 [3. 5. 6. 1. 4. 7. 2. 9. 8.]]


In [111]:
print('For 0: If same result, then 0:\n',result0-sudoko_matrix_solution0)

For 0: If same result, then 0:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [112]:
print('For 10: If same result, then 0:\n',result10-sudoko_matrix_solution10)

For 10: If same result, then 0:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [113]:
print('For 100: If same result, then 0:\n',result100-sudoko_matrix_solution100)

For 100: If same result, then 0:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
